## Load packages and initialize GEE

In [24]:
# import packages
import ee
import geemap
import pandas as pd

In [25]:
# authenticate the EE api
# ee.Authenticate()

In [26]:
# initialize the EE api
ee.Initialize(project='ee-bermane')

## Define GEE datasets

In [53]:
# define EE datasets
# biomass
agb = ee.ImageCollection("projects/sat-io/open-datasets/ESA/ESA_CCI_AGB")
rsr = ee.Image("projects/ee-bermane/assets/Root_shoot_ratio_Map_Merged")

# soc
soc_olm = ee.Image("projects/ee-bermane/assets/soc_0_1m_kg_m2_olm")
soc_sothe = ee.Image(
    "projects/ee-bermane/assets/McMaster_WWFCanada_soil_carbon1m_250m_kg-m2_version3")
soc_0_10_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_0_10cm_kg_m2_olm_global")
soc_10_30_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_10_30cm_kg_m2_olm_global")
soc_30_60_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_30_60cm_kg_m2_olm_global")
soc_60_100_olm_global = ee.Image(
    "projects/ee-bermane/assets/soc_60_100cm_kg_m2_olm_global")

# peatlands
peat = ee.Image("projects/sat-io/open-datasets/GLOBAL-PEATLAND-DATABASE")

# landcover
lc = ee.Image("USGS/NLCD_RELEASES/2020_REL/NALCMS")
wte = ee.Image("projects/ee-bermane/assets/WTE_2020")
k1 = ee.Image("projects/ee-bermane/assets/k1_us_can")

# netflux
netflux = ee.ImageCollection(
    "projects/wri-datalab/gfw-data-lake/net-flux-forest-extent-per-ha-v1-3-2-2001-2023/net-flux-global-forest-extent-per-ha-2001-2023"
)

# vector
y2y = ee.FeatureCollection("projects/ee-bermane/assets/y2y")
y2y_ecoregions = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_ecoregions")
y2y_biomes = ee.FeatureCollection("projects/ee-bermane/assets/y2y_biomes")
rra = ee.FeatureCollection("projects/ee-bermane/assets/ross_river_ipca")
countries = ee.FeatureCollection("USDOS/LSIB/2017")
y2y_protected_areas = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_protected_areas")
y2y_protected_areas_canada = ee.FeatureCollection(
    "projects/ee-bermane/assets/y2y_protected_areas_canada")
us_can = ee.FeatureCollection("projects/ee-bermane/assets/us_can_simple")

## Calculate north america biomass

In [54]:
# calc 2021 ESA CCI biomass
# grab the 2021 AGB images
agb_2021 = agb.filter(ee.Filter.stringContains("system:index", "2021")).first()

# create image for litter using Harris ratio (4%)
# leave out dead wood since it is likely sensed already
# mask litter only for forested LC types
# 1: "#033e00",  # Temperate or sub-polar needleleaf forest
# 2: "#939b71",  # Sub-polar taiga needleleaf forest
# 3: "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
# 4: "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
# 5: "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
# 6: "#6b7d2c",  # Mixed forest

litter_mask = lc.gte(1).And(lc.lte(6))

litter = agb_2021.select(['AGB']).multiply(0.04).multiply(litter_mask)

# create image for BGB (using global rsr map)
bgb = agb_2021.select(['AGB']).multiply(rsr).rename('BGB')

# add AGB + litter and BGB together
bio_2021 = agb_2021.select(['AGB']).add(litter).addBands(
    bgb).rename(['agb_t_ha', 'bgb_t_ha'])

# multiply values by 0.47 to get carbon density
# 0.47 used by Harris et al. (2021)
# mask barren ground, snow/ice and water
bio_2021 = bio_2021.multiply(0.47).updateMask(
    lc.neq(18).And(lc.neq(19)).And(lc.neq(16)))

# compute per-pixel area in ha
pixel_area_ha = ee.Image.pixelArea().divide(10000)

# Create carbon layer mask to filter pixel area raster
carbon_mask = bio_2021.select(['agb_t_ha']).mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_agb_extent = pixel_area_ha.updateMask(carbon_mask)

# calculate total biomass c per pixel and rename bands
bio_stock_2021 = bio_2021.multiply(
    pixel_area_ha
).rename(
    ['agb_t', 'bgb_t']
).addBands(
    pixel_area_agb_extent.rename(['pixel_area_agb_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Calculate soil carbon from Sothe Canada data and Open Land Map US data

In [55]:
# calc open land map soc
# add the layers together from 0-100cm
soc_olm = soc_0_10_olm_global.add(soc_10_30_olm_global).add(
    soc_30_60_olm_global
).add(
    soc_60_100_olm_global
)
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_olm = soc_olm.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# reproject to match sothe to blend images
soc_olm_reproj_sothe = soc_olm.resample('bilinear').toFloat()

In [56]:
# calc sothe soc
# multiply by 10 to get t/ha
# mask water/snow/ice
soc_sothe = soc_sothe.multiply(10).rename(
    'soc_dens').updateMask(lc.neq(18).And(lc.neq(19)))

# blend sothe and olm carbon across y2y
soc_blend = ee.ImageCollection([soc_olm_reproj_sothe, soc_sothe]).mosaic().rename('soc_t_ha').reproject(
    crs=soc_sothe.projection(),
    crsTransform=soc_sothe.projection().getInfo().get('transform')
).updateMask(lc.neq(18).And(lc.neq(19)))  # mask snow/ice, water

# Create soc layer mask to filter pixel area raster
soc_mask = soc_blend.mask().neq(0)

# Mask pixel_area_ha to carbon layers
pixel_area_soc_extent = pixel_area_ha.updateMask(soc_mask)

# multiply by pixel area to get total carbon per pixel
soc_blend_stock = soc_blend.multiply(pixel_area_ha).rename('soc_t').addBands(
    pixel_area_soc_extent.rename(['pixel_area_soc_extent_ha'])
).addBands(
    pixel_area_ha.rename(['pixel_area_ha'])
)

## Calculate average carbon density in mountains vs. non-mountains in US and Canada combined

do I need to mask for values >0 as well to make sure we are fairly counting where there is actually biomass??? or is that already done?

In [63]:
# mask biomass with k1 mountain layer
# reduce to single image
biomass_k1 = (
    bio_2021
    .updateMask(k1) # mask for mountains only
    .reduce(ee.Reducer.sum()) # sum biomass layers
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=us_can.geometry(),
        scale=bio_2021.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
# too big a process to load directly
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, biomass_k1)]),
    description="biomass_k1_mean_density",
    folder="",
    fileFormat="CSV"
).start()


In [64]:
# mask biomass with not k1 mountain layer
# reduce to single image
biomass_not_k1 = (
    bio_2021
    .updateMask(k1.Not()) # mask for not mountains
    .reduce(ee.Reducer.sum()) # sum biomass layers
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=us_can.geometry(),
        scale=bio_2021.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
# too big a process to load directly
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, biomass_not_k1)]),
    description="biomass_not_k1_mean_density",
    folder="",
    fileFormat="CSV"
).start()

In [67]:
# mask soc with k1 mountain layer
# reduce to single image
soc_k1 = (
    soc_blend
    .updateMask(k1) # mask for mountains only
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=us_can.geometry(),
        scale=soc_blend.projection().nominalScale(),
        maxPixels=1e20
    )
)

soc_not_k1 = (
    soc_blend
    .updateMask(k1.Not()) # mask for not mountains
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=us_can.geometry(),
        scale=soc_blend.projection().nominalScale(),
        maxPixels=1e20
    )
)

# export to drive
# too big a process to load directly
ee.batch.Export.table.toDrive(
    collection=ee.FeatureCollection([
        ee.Feature(None, {"soc_k1": soc_k1, "soc_not_k1": soc_not_k1})]),
    description="soc_k1_mean_density",
    folder="",
    fileFormat="CSV"
).start()


In [71]:
# mask biomass k1 for Y2Y only
biomass_k1_y2y = (
    bio_2021
    .updateMask(k1) # mask for mountains only
    .reduce(ee.Reducer.sum()) # sum biomass layers
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=y2y.geometry(),
        scale=bio_2021.projection().nominalScale(),
        maxPixels=1e20
    )
)

biomass_k1_y2y.getInfo()

{'sum': 56.21426028703218}

In [72]:
# mask soc k1 for Y2Y only
soc_k1_y2y = (
    soc_blend
    .updateMask(k1) # mask for mountains only
    .reduceRegion( # calc mean biomass density
        reducer=ee.Reducer.mean(),
        geometry=y2y.geometry(),
        scale=soc_blend.projection().nominalScale(),
        maxPixels=1e20
    )
)

soc_k1_y2y.getInfo()

{'soc_t_ha': 208.6348971535799}

## Calculate average carbon density in different WTE classes in US and Canada combined

## Add layers to map

In [20]:
# set biomass palette
biomass_palette = [
    "#C6ECAE", "#A1D490", "#7CB970", "#57A751", "#348E32",
    "#267A29", "#176520", "#0C4E15", "#07320D", "#031807"
]

# set land cover palette
lc_palette = [
    "#033e00",  # Temperate or sub-polar needleleaf forest
    "#939b71",  # Sub-polar taiga needleleaf forest
    "#196d12",  # Tropical or sub-tropical broadleaf evergreen forest
    "#1fab01",  # Tropical or sub-tropical broadleaf deciduous forest
    "#5b725c",  # Temperate or sub-polar broadleaf deciduous forest
    "#6b7d2c",  # Mixed forest
    "#b29d29",  # Tropical or sub-tropical shrubland
    "#b48833",  # Temperate or sub-polar shrubland
    "#e9da5d",  # Tropical or sub-tropical grassland
    "#e0cd88",  # Temperate or sub-polar grassland
    "#a07451",  # Sub-polar or polar shrubland-lichen-moss
    "#bad292",  # Sub-polar or polar grassland-lichen-moss
    "#3f8970",  # Sub-polar or polar barren-lichen-moss
    "#6ca289",  # Wetland
    "#e6ad6a",  # Cropland
    "#a9abae",  # Barren land
    "#db2126",  # Urban and built-up
    "#4c73a1",  # Water
    "#fff7fe",  # Snow and ice
]

In [69]:
# create a map
m = geemap.Map()

# add layers
# m.addLayer(soc_blend, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Density Blend 0-1m")
# m.addLayer(soc_olm_reproj_sothe, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC OLM Density 0-1m")
# m.addLayer(soc_sothe, {"min": 1, "max": 450, "palette": biomass_palette}, "SOC Sothe Density 0-1m")
# m.addLayer(rsr, {}, 'Root-to-Shoot Ratio Global')
# m.addLayer(bio_2021.select(['agb_t_ha']).clip(y2y), {
#           "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(bio_2021.select(['dpm_t_ha']).clip(y2y), {"min": 1, "max": 60, "palette": biomass_palette}, 'DPM Density')
# m.addLayer(pixel_area_agb_extent.clip(y2y), {}, 'Carbon Mask')
m.addLayer(lc, {"min": 1, "max": 19,
                "palette": lc_palette}, 'Landcover')
m.addLayer(lc.mask(), {}, 'LC Mask')
# m.addLayer(y2y_can, {}, 'Y2Y Canada Extent')
# m.addLayer(big_fc, {}, 'Big Geo')
# m.addLayer(bio_2021.select(['agb_t_ha']).updateMask(k1.Not()), {
#           "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(agb_2021.select(['AGB']), {
#     "min": 1, "max": 100, "palette": biomass_palette}, 'CCI AGB Density')
# m.addLayer(k1, {"min": 0, "max": 1}, 'K1 Mountain Classification')
m.addLayer(wte.updateMask(lc.mask()), {"min": 1, "max": 450, "palette": biomass_palette}, "WTE")

# Display the map
m

Map(center=[0, 0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI(childr…